In [7]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [8]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vaini,TO,2022-02-02 02:56:43,-21.2000,-175.2000,86.16,70,40,10.36
1,1,Albany,US,2022-02-02 02:57:42,42.6001,-73.9662,29.05,71,100,3.00
2,2,Ushuaia,AR,2022-02-02 02:57:43,-54.8000,-68.3000,42.46,65,40,37.98
3,3,Brenham,US,2022-02-02 02:57:43,30.1669,-96.3977,60.98,87,100,0.00
4,4,Port Alfred,ZA,2022-02-02 02:54:12,-33.5906,26.8910,77.49,74,100,7.18


In [13]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [22]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 75


In [27]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Ushuaia,AR,2022-02-02 02:57:43,-54.8000,-68.3000,42.46,65,40,37.98
3,3,Brenham,US,2022-02-02 02:57:43,30.1669,-96.3977,60.98,87,100,0.00
6,6,Odienne,CI,2022-02-02 02:57:44,9.5100,-7.5692,69.75,18,61,4.21
8,8,Saldanha,ZA,2022-02-02 02:57:44,-33.0117,17.9442,66.25,96,78,12.19
9,9,Saint George,US,2022-02-02 02:53:13,37.1041,-113.5841,43.39,24,0,1.99
10,10,Big Spring,US,2022-02-02 02:57:45,32.2504,-101.4787,53.08,32,0,1.99
19,19,Muros,ES,2022-02-02 02:57:48,42.7762,-9.0603,48.70,87,0,5.61
20,20,Umarkot,PK,2022-02-02 02:57:48,25.3616,69.7362,66.20,60,15,10.42
22,22,Bredasdorp,ZA,2022-02-02 02:57:48,-34.5322,20.0403,71.73,80,8,4.03
25,25,Castro,BR,2022-02-02 02:57:49,-24.7911,-50.0119,62.37,96,74,5.21


In [30]:
preferred_cities_df.count()


City_ID       245
City          245
Country       240
Date          245
Lat           245
Lng           245
Max Temp      245
Humidity      245
Cloudiness    245
Wind Speed    245
dtype: int64

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Ushuaia,AR,42.46,-54.8000,-68.3000,
3,Brenham,US,60.98,30.1669,-96.3977,
6,Odienne,CI,69.75,9.5100,-7.5692,
8,Saldanha,ZA,66.25,-33.0117,17.9442,
9,Saint George,US,43.39,37.1041,-113.5841,
10,Big Spring,US,53.08,32.2504,-101.4787,
19,Muros,ES,48.70,42.7762,-9.0603,
20,Umarkot,PK,66.20,25.3616,69.7362,
22,Bredasdorp,ZA,71.73,-34.5322,20.0403,
25,Castro,BR,62.37,-24.7911,-50.0119,


In [32]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [33]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}

In [34]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [38]:
# Grab the first hotel from the results and store the name.
  try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
  except (IndexError):
    print("Hotel not found... skipping.")

IndentationError: unexpected indent (1383625399.py, line 2)

In [39]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [42]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [43]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [44]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))